In [ ]:

# Transformer 모델을 위한 설정
class TransformerModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_heads=2, num_layers=4, dropout=0.1):
        super(TransformerModel, self).__init__()
        
        # Multi-Head Attention 레이어
        self.attention = torch.nn.MultiheadAttention(embed_dim=input_size, num_heads=num_heads, dropout=dropout)
        
        # Transformer Encoder
        self.transformer = torch.nn.TransformerEncoder(
            torch.nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads, dim_feedforward=hidden_size, dropout=dropout), 
            num_layers=num_layers
        )
        
        # Fully connected layers
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        # 시퀀스 길이, 배치 크기, 특성 차원에 맞게 변환
        x = x.transpose(0, 1)  # Transformer는 (seq_len, batch_size, features)의 형태를 기대함
        
        # Attention 통과
        attn_output, _ = self.attention(x, x, x)
        
        # Transformer Encoder 통과
        transformer_output = self.transformer(attn_output)
        
        # 마지막 시퀀스 출력을 사용 (기본적으로 클래스 레이블 예측)
        output = transformer_output[-1, :, :]
        
        # Fully connected layers 통과
        output = self.fc(output)
        
        return output
